# Interactive plot with `fastplotlib`

In [1]:
import fastplotlib as fpl
import imageio.v3 as iio
from matplotlib import pyplot as plt

from movement import sample_data

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Available devices:
🯄 (default) | Apple M2 | IntegratedGPU | Metal | 


SHA256 hash of downloaded file: b4f08dce448efe05068284652f936b7e37e6f80a893d27aa2c59cd0aa8b15f8b
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


## Fetch a sample dataset 
Three mice moving into a circular arena.

First let's load the pose tracks.

In [2]:
ds = sample_data.fetch_dataset(
    "SLEAP_three-mice_Aeon_proofread.analysis.h5",
    with_video=True,
)
ds

<xarray.Dataset> Size: 27kB
Dimensions:      (time: 601, individuals: 3, keypoints: 1, space: 2)
Coordinates:
  * time         (time) float64 5kB 0.0 0.02 0.04 0.06 ... 11.96 11.98 12.0
  * individuals  (individuals) <U10 120B 'AEON3B_NTP' 'AEON3B_TP1' 'AEON3B_TP2'
  * keypoints    (keypoints) <U8 32B 'centroid'
  * space        (space) <U1 8B 'x' 'y'
Data variables:
    position     (time, individuals, keypoints, space) float32 14kB 770.3 ......
    confidence   (time, individuals, keypoints) float32 7kB nan nan ... nan nan
Attributes:
    fps:              50.0
    time_unit:        seconds
    source_software:  SLEAP
    source_file:      /Users/nsirmpilatze/.movement/data/poses/SLEAP_three-mi...
    frame_path:       /Users/nsirmpilatze/.movement/data/frames/three-mice_Ae...
    video_path:       /Users/nsirmpilatze/.movement/data/videos/three-mice_Ae...

Data wrangling: create a list of tracks arrays (one per mouse)

In [3]:
ind_tracks = []
for ind in ds.individuals.values:
    track = ds["position"].sel(individuals=ind, keypoints="centroid").values
    ind_tracks.append(track)

Next, let's load the video

In [4]:
video = iio.imread(ds.video_path, index=None, plugin="pyav")[:, :, :, 0]

In [5]:
n_frames, height, width = video.shape
print(f"Video shape: n_frames {n_frames}, height {height}, width {width}")

Video shape: n_frames 601, height 1080, width 1440


## Create the interactive figure

In [6]:
movie = fpl.ImageWidget(data=video, cmap="gray", histogram_widget=False)

fig = movie.figure
current_t = movie.current_index["t"]

# line to display the behavior tracks
palette = ["#1b9e77", "#d95f02", "#7570b3"]
fig[0,0].add_line_collection(
    ind_tracks,
    colors=palette,
    thickness=2,
    alpha=0.5,
    names=ds.individuals.values
)

fig[0, 0].auto_scale()

movie.show()

RFBOutputContext()

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…